In [1]:
import gradio as gr
import random
import os
import csv
from datetime import datetime, timezone, timedelta

directories = ['vae_inference', 'gan_inference', 'vaegan_inference', 'validation_audio', 'vae_inference_peak', "audio_from_peak_processed"]
files = {dir: [os.path.join(dir, f) for f in os.listdir(dir) if f.endswith('.wav')] for dir in directories}

csv_file = "votes.csv"

if not os.path.exists(csv_file):
    with open(csv_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["audio1", "audio2", "audio1 wins", "time", "id"])

def get_audio_pair():
    dir1, dir2 = random.sample(directories, 2)
    file1, file2 = random.choice(files[dir1]), random.choice(files[dir2])
    return file1, file2

def fn(current_audio):
    a1, a2 = get_audio_pair()
    return f"{random.randint(0, 1e16):016d}", a1, a2, (a1, a2)

def test_fn(btn, user_id, current_audio):
    print("here")
    with open(csv_file, 'a', newline='') as f:
        writer = csv.writer(f)
        now = datetime.now(timezone.utc).astimezone(timezone(timedelta(hours=1)))
        now_str = now.strftime('%d/%m/%Y %H:%M:%S')
        writer.writerow([*current_audio, "1" in btn, now_str, user_id])
    a1, a2 = get_audio_pair()
    return a1, a2, (a1, a2)

with gr.Blocks(title="Audio eval") as demo:
    current_audio = gr.State()
    gr.Markdown(
    """
    # Evaluation de sons d'oiseaux générés
    Choisir l'audio qui ressemble le plus à un chant d'oiseau
    """)
    with gr.Row(equal_height=True):
        a1, a2 = get_audio_pair()
        with gr.Column(scale=1):
            audio1 = gr.Audio(a1, label="Audio1")
            btn1 = gr.Button(value="Audio1 est meilleur")
        with gr.Column(scale=1):
            audio2 = gr.Audio(a2, label="Audio2")
            btn2 = gr.Button(value="Audio2 est meilleur")
    user_id = gr.Text(label="user_id")
    demo.load(fn, [current_audio], [user_id, audio1, audio2, current_audio])
    btn1.click(test_fn, [btn1, user_id, current_audio], [audio1, audio2, current_audio])
    btn2.click(test_fn, [btn2, user_id, current_audio], [audio1, audio2, current_audio])

demo.launch(share=False, server_name="0.0.0.0", server_port=7860)

Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.
